In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
from numpy import percentile
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
hist_trans_df1 = pd.read_csv('../input/elo-cards-basic-eda-exposes-data-skewness/hist_DecFeb18.csv')

In [ ]:
hist_trans_df2 = pd.read_csv('../input/elo-cards-basic-eda-exposes-data-skewness/hist_SepNov.csv')

In [ ]:
new_trans_df=pd.read_csv('../input/elo-merchant-category-recommendation/new_merchant_transactions.csv')

In [ ]:
trans_df = pd.concat([hist_trans_df1,hist_trans_df2,new_trans_df])

In [ ]:
train_df = pd.read_csv('../input/elo-merchant-category-recommendation/train.csv')

In [ ]:
test_df = pd.read_csv('../input/elo-merchant-category-recommendation/test.csv')

In [ ]:
train_df_merged = pd.merge(trans_df, train_df, how = 'inner', left_on = ['card_id'], right_on=['card_id'])

In [ ]:
train_df_merged['target'].hist(bins=50)

In [ ]:
print(train_df_merged['target'].max())
print(train_df_merged['target'].min())
print(train_df_merged['target'].median())

In [ ]:
train_df_lt2 = train_df_merged.loc[(train_df_merged.target >= -15) & (train_df_merged.target <= -1)]
train_df_gt2 = train_df_merged.loc[(train_df_merged.target >= 1)]
train_df_0_2 = train_df_merged.loc[(train_df_merged.target > 0) & (train_df_merged.target < 1)]
train_df_2_0 = train_df_merged.loc[(train_df_merged.target > -1) & (train_df_merged.target <= 0)]
train_df_lt15 = train_df_merged.loc[(train_df_merged.target < -15)]

In [ ]:
print(len(set(train_df_merged['card_id'].unique()) & set(hist_trans_df1['card_id'].unique())))

In [ ]:
print(len(set(hist_trans_df1['card_id'].unique()) & set(test_df['card_id'].unique())))

In [ ]:
def prepare_stratifiedByMerchCat(train_df_red,train_df):
    Trans_by_MerchantCat = train_df_red.groupby(['merchant_category_id']).size().reset_index()
    Trans_by_MerchantCat = Trans_by_MerchantCat.rename(columns={0: 'TransCtbyMerchCat'})
    print("Max of Count of Trans by Merch Cat ID:"+str(Trans_by_MerchantCat['TransCtbyMerchCat'].max().round(2)))
    print("Median of Count of Trans by Merch Cat ID:"+str(round(Trans_by_MerchantCat['TransCtbyMerchCat'].median(),2)))
    print("Mean of Count of Trans by Merch Cat ID:"+str(round(Trans_by_MerchantCat['TransCtbyMerchCat'].mean(),2)))
    print("Min of Count of Trans by Merch Cat ID:"+str(round(Trans_by_MerchantCat['TransCtbyMerchCat'].min(),2)))
    data = Trans_by_MerchantCat['TransCtbyMerchCat']
    percentiles = percentile(data,[50,75,90,95,98,99,99.5])
    print(percentiles.round(2))
    list_MerchCat_P995 = Trans_by_MerchantCat['merchant_category_id'].loc[(Trans_by_MerchantCat.TransCtbyMerchCat>percentiles[6])]
    list_MerchCat_P99 = Trans_by_MerchantCat['merchant_category_id'].loc[(Trans_by_MerchantCat.TransCtbyMerchCat>percentiles[5]) & (Trans_by_MerchantCat.TransCtbyMerchCat<=percentiles[6])]
    list_MerchCat_P98 = Trans_by_MerchantCat['merchant_category_id'].loc[(Trans_by_MerchantCat.TransCtbyMerchCat>percentiles[4]) & (Trans_by_MerchantCat.TransCtbyMerchCat<=percentiles[5])]
    list_MerchCat_P95 = Trans_by_MerchantCat['merchant_category_id'].loc[(Trans_by_MerchantCat.TransCtbyMerchCat>percentiles[3]) & (Trans_by_MerchantCat.TransCtbyMerchCat<=percentiles[4])]
    list_MerchCat_P90 = Trans_by_MerchantCat['merchant_category_id'].loc[(Trans_by_MerchantCat.TransCtbyMerchCat>percentiles[2]) & (Trans_by_MerchantCat.TransCtbyMerchCat<=percentiles[3])]
    list_MerchCat_P75 = Trans_by_MerchantCat['merchant_category_id'].loc[(Trans_by_MerchantCat.TransCtbyMerchCat>percentiles[1]) & (Trans_by_MerchantCat.TransCtbyMerchCat<=percentiles[2])]
    list_MerchCat_P50 = Trans_by_MerchantCat['merchant_category_id'].loc[(Trans_by_MerchantCat.TransCtbyMerchCat>percentiles[0]) & (Trans_by_MerchantCat.TransCtbyMerchCat<=percentiles[1])]
    list_MerchCat_Plt50 = Trans_by_MerchantCat['merchant_category_id'].loc[(Trans_by_MerchantCat.TransCtbyMerchCat<=percentiles[0])]
    print("P99.5 Merch Cat:")
    print(len(list_MerchCat_P995))
    trans_by_merch_cat=train_df_red.groupby(['card_id','merchant_category_id']).size().reset_index()
    amt_by_merch_cat=train_df_red.groupby(['card_id','merchant_category_id'])['purchase_amount'].sum().reset_index()
    
    trans_by_merch_cat = trans_by_merch_cat.rename(columns={0: 'TransCtbyMerchCat'})
    amt_by_merch_cat = amt_by_merch_cat.rename(columns={'purchase_amount': 'AmtbyMerchCat'})
    
    trans_by_merch_cat_P995 = trans_by_merch_cat.loc[(trans_by_merch_cat.merchant_category_id.isin(list_MerchCat_P995))]
    trans_by_merch_cat_P99 = trans_by_merch_cat.loc[(trans_by_merch_cat.merchant_category_id.isin(list_MerchCat_P99))]
    trans_by_merch_cat_P98 = trans_by_merch_cat.loc[(trans_by_merch_cat.merchant_category_id.isin(list_MerchCat_P98))]
    trans_by_merch_cat_P95 = trans_by_merch_cat.loc[(trans_by_merch_cat.merchant_category_id.isin(list_MerchCat_P95))]
    trans_by_merch_cat_P90 = trans_by_merch_cat.loc[(trans_by_merch_cat.merchant_category_id.isin(list_MerchCat_P90))]
    trans_by_merch_cat_P75 = trans_by_merch_cat.loc[(trans_by_merch_cat.merchant_category_id.isin(list_MerchCat_P75))]
    trans_by_merch_cat_P50 = trans_by_merch_cat.loc[(trans_by_merch_cat.merchant_category_id.isin(list_MerchCat_P50))]
    trans_by_merch_cat_Plt50 = trans_by_merch_cat.loc[(trans_by_merch_cat.merchant_category_id.isin(list_MerchCat_Plt50))]
    
    amt_by_merch_cat_P995 = amt_by_merch_cat.loc[(amt_by_merch_cat.merchant_category_id.isin(list_MerchCat_P995))]
    amt_by_merch_cat_P99 = amt_by_merch_cat.loc[(amt_by_merch_cat.merchant_category_id.isin(list_MerchCat_P99))]
    amt_by_merch_cat_P98 = amt_by_merch_cat.loc[(amt_by_merch_cat.merchant_category_id.isin(list_MerchCat_P98))]
    amt_by_merch_cat_P95 = amt_by_merch_cat.loc[(amt_by_merch_cat.merchant_category_id.isin(list_MerchCat_P95))]
    amt_by_merch_cat_P90 = amt_by_merch_cat.loc[(amt_by_merch_cat.merchant_category_id.isin(list_MerchCat_P90))]
    amt_by_merch_cat_P75 = amt_by_merch_cat.loc[(amt_by_merch_cat.merchant_category_id.isin(list_MerchCat_P75))]
    amt_by_merch_cat_P50 = amt_by_merch_cat.loc[(amt_by_merch_cat.merchant_category_id.isin(list_MerchCat_P50))]
    amt_by_merch_cat_Plt50 = amt_by_merch_cat.loc[(amt_by_merch_cat.merchant_category_id.isin(list_MerchCat_Plt50))]
    
    trans_by_merch_cat_P995_card = trans_by_merch_cat_P995.groupby(['card_id'])['TransCtbyMerchCat'].sum().reset_index()
    trans_by_merch_cat_P99_card = trans_by_merch_cat_P99.groupby(['card_id'])['TransCtbyMerchCat'].sum().reset_index()
    trans_by_merch_cat_P98_card = trans_by_merch_cat_P98.groupby(['card_id'])['TransCtbyMerchCat'].sum().reset_index()
    trans_by_merch_cat_P95_card = trans_by_merch_cat_P95.groupby(['card_id'])['TransCtbyMerchCat'].sum().reset_index()
    trans_by_merch_cat_P90_card = trans_by_merch_cat_P90.groupby(['card_id'])['TransCtbyMerchCat'].sum().reset_index()
    trans_by_merch_cat_P75_card = trans_by_merch_cat_P75.groupby(['card_id'])['TransCtbyMerchCat'].sum().reset_index()
    trans_by_merch_cat_P50_card = trans_by_merch_cat_P50.groupby(['card_id'])['TransCtbyMerchCat'].sum().reset_index()
    trans_by_merch_cat_Plt50_card = trans_by_merch_cat_Plt50.groupby(['card_id'])['TransCtbyMerchCat'].sum().reset_index()
    
    amt_by_merch_cat_P995_card = amt_by_merch_cat_P995.groupby(['card_id'])['AmtbyMerchCat'].sum().reset_index()
    amt_by_merch_cat_P99_card = amt_by_merch_cat_P99.groupby(['card_id'])['AmtbyMerchCat'].sum().reset_index()
    amt_by_merch_cat_P98_card = amt_by_merch_cat_P98.groupby(['card_id'])['AmtbyMerchCat'].sum().reset_index()
    amt_by_merch_cat_P95_card = amt_by_merch_cat_P95.groupby(['card_id'])['AmtbyMerchCat'].sum().reset_index()
    amt_by_merch_cat_P90_card = amt_by_merch_cat_P90.groupby(['card_id'])['AmtbyMerchCat'].sum().reset_index()
    amt_by_merch_cat_P75_card = amt_by_merch_cat_P75.groupby(['card_id'])['AmtbyMerchCat'].sum().reset_index()
    amt_by_merch_cat_P50_card = amt_by_merch_cat_P50.groupby(['card_id'])['AmtbyMerchCat'].sum().reset_index()
    amt_by_merch_cat_Plt50_card = amt_by_merch_cat_Plt50.groupby(['card_id'])['AmtbyMerchCat'].sum().reset_index()
    
    trans_by_merch_cat_P995_card = trans_by_merch_cat_P995_card.rename(columns={'TransCtbyMerchCat': 'TransCtbyMerchCatP99.5'})
    trans_by_merch_cat_P99_card = trans_by_merch_cat_P99_card.rename(columns={'TransCtbyMerchCat': 'TransCtbyMerchCatP99'})
    trans_by_merch_cat_P98_card = trans_by_merch_cat_P98_card.rename(columns={'TransCtbyMerchCat': 'TransCtbyMerchCatP98'})
    trans_by_merch_cat_P95_card = trans_by_merch_cat_P95_card.rename(columns={'TransCtbyMerchCat': 'TransCtbyMerchCatP95'})
    trans_by_merch_cat_P90_card = trans_by_merch_cat_P90_card.rename(columns={'TransCtbyMerchCat': 'TransCtbyMerchCatP90'})
    trans_by_merch_cat_P75_card = trans_by_merch_cat_P75_card.rename(columns={'TransCtbyMerchCat': 'TransCtbyMerchCatP75'})
    trans_by_merch_cat_P50_card = trans_by_merch_cat_P50_card.rename(columns={'TransCtbyMerchCat': 'TransCtbyMerchCatP50'})
    trans_by_merch_cat_Plt50_card = trans_by_merch_cat_Plt50_card.rename(columns={'TransCtbyMerchCat': 'TransCtbyMerchCatPlt50'})
    
    amt_by_merch_cat_P995_card = amt_by_merch_cat_P995_card.rename(columns={'AmtbyMerchCat': 'AmtbyMerchCatP99.5'})
    amt_by_merch_cat_P99_card = amt_by_merch_cat_P99_card.rename(columns={'AmtbyMerchCat': 'AmtbyMerchCatP99'})
    amt_by_merch_cat_P98_card = amt_by_merch_cat_P98_card.rename(columns={'AmtbyMerchCat': 'AmtbyMerchCatP98'})
    amt_by_merch_cat_P95_card = amt_by_merch_cat_P95_card.rename(columns={'AmtbyMerchCat': 'AmtbyMerchCatP95'})
    amt_by_merch_cat_P90_card = amt_by_merch_cat_P90_card.rename(columns={'AmtbyMerchCat': 'AmtbyMerchCatP90'})
    amt_by_merch_cat_P75_card = amt_by_merch_cat_P75_card.rename(columns={'AmtbyMerchCat': 'AmtbyMerchCatP75'})
    amt_by_merch_cat_P50_card = amt_by_merch_cat_P50_card.rename(columns={'AmtbyMerchCat': 'AmtbyMerchCatP50'})
    amt_by_merch_cat_Plt50_card = amt_by_merch_cat_Plt50_card.rename(columns={'AmtbyMerchCat': 'AmtbyMerchCatPlt50'})
    print(len(trans_by_merch_cat_P99_card))
    print(len(trans_by_merch_cat_P995_card))
    train_df_merged = train_df.merge(trans_by_merch_cat_P995_card, how='left', on = ['card_id'])
    train_df_merged = train_df_merged.merge(trans_by_merch_cat_P99_card, how='left', on = ['card_id'])
    train_df_merged = train_df_merged.merge(trans_by_merch_cat_P98_card, how='left', on = ['card_id'])
    train_df_merged = train_df_merged.merge(trans_by_merch_cat_P95_card, how='left', on = ['card_id'])
    train_df_merged = train_df_merged.merge(trans_by_merch_cat_P90_card, how='left', on = ['card_id'])
    train_df_merged = train_df_merged.merge(trans_by_merch_cat_P75_card, how='left', on = ['card_id'])
    train_df_merged = train_df_merged.merge(trans_by_merch_cat_P50_card, how='left', on = ['card_id'])
    train_df_merged = train_df_merged.merge(trans_by_merch_cat_Plt50_card, how='left', on = ['card_id'])
    
    train_df_merged = train_df_merged.merge(amt_by_merch_cat_P995_card, how='left', on = ['card_id'])
    train_df_merged = train_df_merged.merge(amt_by_merch_cat_P99_card, how='left', on = ['card_id'])
    train_df_merged = train_df_merged.merge(amt_by_merch_cat_P98_card, how='left', on = ['card_id'])
    train_df_merged = train_df_merged.merge(amt_by_merch_cat_P95_card, how='left', on = ['card_id'])
    train_df_merged = train_df_merged.merge(amt_by_merch_cat_P90_card, how='left', on = ['card_id'])
    train_df_merged = train_df_merged.merge(amt_by_merch_cat_P75_card, how='left', on = ['card_id'])
    train_df_merged = train_df_merged.merge(amt_by_merch_cat_P50_card, how='left', on = ['card_id'])
    train_df_merged = train_df_merged.merge(amt_by_merch_cat_Plt50_card, how='left', on = ['card_id'])
    train_df_merged.fillna(0, inplace=True)
    print(" df columns:")
    print(train_df_merged.columns)
    #print("df target range")
    #print(train_df_red['target'].max())
    #print(train_df_red['target'].min())
    print(" df length")
    print(len(train_df_merged))
  
    return train_df_merged

In [ ]:
train_df_merged = prepare_stratifiedByMerchCat(train_df_lt2,train_df)
train_df_merged.to_csv('train_df_strat_MerchCat_lt2.csv')
train_df_merged = prepare_stratifiedByMerchCat(train_df_gt2,train_df)
train_df_merged.to_csv('train_df_strat_MerchCat_gt2.csv')
train_df_merged = prepare_stratifiedByMerchCat(train_df_0_2,train_df)
train_df_merged.to_csv('train_df_strat_MerchCat_0_2.csv')
train_df_merged = prepare_stratifiedByMerchCat(train_df_2_0,train_df)
train_df_merged.to_csv('train_df_strat_MerchCat_2_0.csv')
train_df_merged = prepare_stratifiedByMerchCat(train_df_lt15,train_df)
train_df_merged.to_csv('train_df_strat_MerchCat_lt15.csv')


In [ ]:
test_df_merged = prepare_stratifiedByMerchCat(trans_df,test_df)

In [ ]:
test_df_merged.to_csv('test_df_strat_MerchCat.csv')